In [ ]:
# data analysis libraries:
import numpy as np
import pandas as pd

# data visualization libraries:
import matplotlib.pyplot as plt
import matplotlib.pylab as plt
import seaborn as sns

# to ignore warnings:
import warnings
warnings.filterwarnings('ignore')

# to display all columns:
pd.set_option('display.max_columns', None)
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score
import matplotlib.gridspec as gridspec
%matplotlib inline

# Standard plotly imports

import plotly.graph_objs as go
import plotly.tools as tls
from plotly.offline import iplot, init_notebook_mode
import cufflinks
import cufflinks as cf
import plotly.figure_factory as ff
import os
print(os.listdir("../input"))
plt.style.use('ggplot')
color_pal = [x['color'] for x in plt.rcParams['axes.prop_cycle']]





In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy as sp
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

# Standard plotly imports
#import plotly.plotly as py
import plotly.graph_objs as go
import plotly.tools as tls
from plotly.offline import iplot, init_notebook_mode
#import cufflinks
#import cufflinks as cf
import plotly.figure_factory as ff

# Using plotly + cufflinks in offline mode
init_notebook_mode(connected=True)
#cufflinks.go_offline(connected=True)

# Preprocessing, modelling and evaluating
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn.model_selection import StratifiedKFold, cross_val_score, KFold
from xgboost import XGBClassifier
import xgboost as xgb

## Hyperopt modules
from hyperopt import fmin, hp, tpe, Trials, space_eval, STATUS_OK, STATUS_RUNNING
from functools import partial

import os
import gc
print(os.listdir("../input"))

In [ ]:
# Read train and test data with pd.read_csv():
train_id= pd.read_csv("/kaggle/input/ieee-fraud-detection/train_identity.csv")
test_id = pd.read_csv("/kaggle/input/ieee-fraud-detection/test_identity.csv")
train_tr = pd.read_csv("/kaggle/input/ieee-fraud-detection/train_transaction.csv")
test_tr = pd.read_csv("/kaggle/input/ieee-fraud-detection/test_transaction.csv")

In [ ]:
## Function to reduce the DF size
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
## REducing memory
train_tr = reduce_mem_usage(train_tr)
train_id = reduce_mem_usage(train_id)

In [ ]:
train_id.head()


In [ ]:
test_id.head()

In [ ]:
train_tr.head()

In [ ]:
test_tr.head()

In [ ]:
train_id.info()

In [ ]:
test_id.info()

In [ ]:
train_tr.info()

In [ ]:
test_tr.info()

In [ ]:
print('train_transaction shape is {}'.format(train_tr.shape))
print('test_transaction shape is {}'.format(test_tr.shape))
print('train_identity shape is {}'.format(train_id.shape))
print('test_identity shape is {}'.format(test_id.shape))

1st problem: NaN
Remember

Not all transactions have corresponding identity information

# train_transaction

In [ ]:
missing_values_count = train_tr.isnull().sum()
print (missing_values_count[0:10])
total_cells = np.product(train_tr.shape)
total_missing = missing_values_count.sum()
print ("% of missing data = ",(total_missing/total_cells) * 100)

> Missing Values in train_tr DataFrame. % 41 percent data are missing

# train_identity

In [ ]:
missing_values_count = train_id.isnull().sum()
print (missing_values_count[0:10])
total_cells = np.product(train_id.shape)
total_missing = missing_values_count.sum()
print ("% of missing data = ",(total_missing/total_cells) * 100)

Missing Values in train_id DataFrame. % 35 percent data are missing

In [ ]:
# Here we confirm that all of the transactions in `train_identity`
print(np.sum(train_tr['TransactionID'].isin(train_id['TransactionID'].unique())))
print(np.sum(test_tr['TransactionID'].isin(test_id['TransactionID'].unique())))

24.4% of TransactionIDs in train (144233 / 590540) have an associated train_identity.
28.0% of TransactionIDs in test (144233 / 590540) have an associated train_identity.

Distribution of Target in Training Set
3.5% of transacations are fraud

In [ ]:
print('  {:.4f}% of Transactions that are fraud in train '.format(train_tr['isFraud'].mean() * 100))

In [ ]:
train_tr.groupby('isFraud') \
    .count()['TransactionID'] \
    .plot(kind='barh',
          title='Is Fraud?',
          figsize=(15, 3))
plt.show()


# **TransactionAmt**
The ammount of transaction. I've taken a log transform in some of these plots to better show the distribution

In [ ]:
train_tr['TransactionAmt'] \
    .apply(np.log) \
    .plot(kind='hist',
          bins=100,
          figsize=(15, 5),
          title='Distribution of Log Transaction Amt')
plt.show()

Fraudulent charges appear to have a higher average transaction ammount

Mean transaction amt for fraud is **149.2444**
Mean transaction amt for non-fraud is **134.5119**

In [ ]:
print('Mean transaction amt for fraud is {:.4f}'.format(train_tr.loc[train_tr['isFraud'] == 1]['TransactionAmt'].mean()))
print('Mean transaction amt for non-fraud is {:.4f}'.format(train_tr.loc[train_tr['isFraud'] == 0]['TransactionAmt'].mean()))

# ProductCD
For now we don't know exactly what these values represent.
W has the most number of observations, C the least.
ProductCD C has the most fraud with >11%
ProductCD W has the least with ~2%

In [ ]:
train_tr.groupby('ProductCD') \
    ['TransactionID'].count() \
    .sort_index() \
    .plot(kind='barh',
          figsize=(20, 5),
         title='Count of Observations by ProductCD')
plt.show()

In [ ]:
train_tr.groupby('ProductCD')['isFraud'] \
    .mean() \
    .sort_index() \
    .plot(kind='barh',
          figsize=(15, 3),
         title='Percentage of Fraud by ProductCD')
plt.show()


# card1 - card6
We are told these are all categorical, even though some appear numeric.

In [ ]:
card_cols = [c for c in train_tr.columns if 'card' in c]
train_tr[card_cols].head()

In [ ]:
color_idx = 0
for c in card_cols:
    if train_tr[c].dtype in ['float64','int64']:
        train_tr[c].plot(kind='hist',
                                      title=c,
                                      bins=50,
                                      figsize=(15, 2),
                                      color=color_pal[color_idx])
    color_idx += 1
    plt.show()
    

In [ ]:
train_tr.head()

#Eksik Veri Yapısının Görselleştirilmesi
!pip install missingno
import missingno as msno
msno.bar(df);
msno.matrix(df);
df
import seaborn as sns
df = sns.load_dataset('planets')
df.head()
df.isnull().sum()
msno.matrix(df);
msno.heatmap(df);
msno.heatmap(df);

In [ ]:
train=pd.merge(train_tr, train_id, on = "TransactionID",how='left',left_index=True, right_index=True)
train.head()

In [ ]:
test=pd.merge(test_tr, test_id, on = "TransactionID",how='left',left_index=True, right_index=True)
test.head()

In [ ]:
del train_id, train_tr, test_id, test_tr

In [ ]:
!pip install missingno
import missingno as msno

In [ ]:
msno.matrix(train.iloc[:,:46]);

In [ ]:
msno.matrix(train.iloc[:,40:80]);

In [ ]:
msno.matrix(train.iloc[:,80:120]);

In [ ]:
msno.matrix(train.iloc[:,120:160]);

In [ ]:
msno.matrix(train.iloc[:,160:200]);

In [ ]:
msno.matrix(train.iloc[:,200:240]);

In [ ]:
msno.matrix(train.iloc[:,240:280]);

In [ ]:
msno.matrix(train.iloc[:,280:320]);

In [ ]:
msno.matrix(train.iloc[:,320:360]);

In [ ]:
msno.matrix(train.iloc[:,360:400]);